# Introduction to Creating Blockchain Transactions
A Code Along to understand how to create basic blockchain transactions. We will use a publicly hosted blockchain node to access the Ethereum network and create and broadcast a simple ETH transfer transaction. 

**Author: Harish Raisinghani, DevRel @ Covalent**. https://twitter.com/harish_yvr

## Web3.py Setup

We will be using [Web3.py](https://web3py.readthedocs.io/en/stable/quickstart.html), a comprehensive Python library for interacting with the programmable [Ethereum](https://ethereum.org/) blockchain, in this Code Along. 

We install the library using `pip`:

In [ ]:
pip install web3

## Connect to a Hosted Node Provider

In order to write data to blockchains, you can either:

1. Run your own node on that blockchain, download the entire ledger, execute the transactions and send data through your node. 

2. Use a hosted node provider who manage the infrastructure for you

> Deeper dive: Set up your own Ethereum node following [this guide.](https://ethereum.org/en/developers/docs/nodes-and-clients/run-a-node/)

Examples of hosted Ethereum node providers are listed here: https://ethereumnodes.com/ . This is just a small set but does cover many of the major providers. 

In this Code Along, we will use the [Pocket Network](https://pokt.network/
 ) hosted node provider which offers a comprehensive [list of public nodes](https://docs.pokt.network/use/public-rpc/). Note the URL for accessing the Ethereum Goerli testnet node: https://eth-goerli-rpc.gateway.pokt.network

We are using a testnet because writing transactions to blockchains requires gas which you need to pay for using the blockchain's native token (e.g. ETH). To use a testnet, we can acquire the corresponding testnet tokens from a faucet for free. 

 The following code configures our `web3.py` library to use the Pocket Network as our node provider on the Ethereum Goerli testnet and confirms if the node is connected to the testnet:

In [82]:
from web3 import Web3

eth_goerli_provider = 'https://eth-goerli-rpc.gateway.pokt.network'

w3_testnet = Web3(Web3.HTTPProvider(eth_goerli_provider))

# Check that it is connected
w3_testnet.isConnected()

True

And let us check the blockchain's chain ID and current block number and confirm it with the Ethereum Goerli block explorer: https://goerli.etherscan.io

In [83]:
w3_testnet.eth.chain_id

5

In [84]:
w3_testnet.eth.get_block_number()

7635987

Now that we are connected to the Goerli testnet, we need to do the following to get set up in order to write transactions to the blockchain:

1. Have a wallet on the Goerli testnet
2. Have some test ETH tokens in our wallet on the Goerli testnet

## Wallet Setup

We can repurpose the code from the previous Code Along to create our wallet.

**NOTE: EVERYTHING WE DO HERE IS FOR DEMO PURPOSES SO PLEASE DO NOT USE ANY WALLETS CREATED HERE AS YOUR PRODUCTION WALLET!**

In [85]:
import secrets
from eth_account import Account

# 1. Create our 32 bytes random number. Each byte converts to two hex digits
priv = secrets.token_hex(32)

# 2. Prefix the '0x' hexadecimal identifier
private_key = "0x" + priv

# 3. Generate an account object that includes the public key and other useful methods
account = Account.from_key(private_key)

# Extract the public key
public_address = account.address

In [86]:
# Print the public key
print("Wallet address is:", public_address)

Wallet address is: 0x3922C053bC7885B20F7550517E47BfCc692120Cc


Next, use a faucet such as https://goerli-faucet.pk910.de/ to 'drip' yourself some test ETH tokens on Goerli. Note for this particular faucet, you need to enter your wallet address and let the faucet run for a few minutes to accumulate the minimum amount of test tokens (0.05 Goerli ETH). 

Verify that you now indeed have some test tokens on Goerli:

In [179]:
w3_testnet.eth.get_balance(public_address)

142724999908650000

Now we can create our first transaction!

The process of creating a transaction involves using the above components to:
1. Create the transaction object
2. Sign the transaction with your private key
3. Send the transaction using the provided node

## Create a Transaction

The following is a very basic transaction object to send 0.001 Goerli ETH to wallet `0xc43ee4634Aed3796971ccE43e64b696D4de4ca83`:



In [178]:
txn_obj = {
    'nonce': w3_testnet.eth.get_transaction_count(public_address),
    'to': '0xc43ee4634Aed3796971ccE43e64b696D4de4ca83',
    'from': public_address,
    'value': w3_testnet.toWei(0.01, 'ether'),
    'data': b'',
    'chainId': w3_testnet.eth.chain_id
}

where we have used the following keys:

- `nonce`: a sequence number issued by the sending Ethereum account indicating the number of previous transactions to prevent message replay
- `from`: the sending Ethereum account
- `to`: the receiving Ethereum account
- `value`: the amount of ETH transferred (in Wei)
- `data`: any data sent with the transaction in *binary*
- `chainId`: the unique integer identifier for Ethereum-compatible blockchain networks

### Gas & Fees

![Ethereum gas fees](https://images.ctfassets.net/0idwgenf7ije/4s9bNHrnl6btZgfwC3Tgee/82c3fba7184d974c13a01c6f5fe71287/What_Are_Ethereum_Gas_Fees?fm=webp)

Now we need to add the keys relating to the amount of gas and the gas price we want to pay for this transaction to our `txn_obj`. 

> Deeper dive: Learn all the nuances of [gas fees](https://ethereum.org/en/developers/docs/gas/)

After the implementation of [EIP-1559](https://ethereum.org/en/developers/docs/gas/#eip-1559), the gas fee structure is more complex but essentially boils down to the following:

- To transfer ETH between accounts, the transaction requires a fixed amount of 21,000 gas units. 

- The fee per gas unit is made up of two components:
  1. A `base fee` which is set by the network and burned
  2. A `priority fee` or tip given to the validator

Let's say Jordan has to pay Taylor 1 ETH. In the transaction, the gas limit is 21,000 units and the base fee is 10 gwei (10^9 wei). Jordan includes a tip of 2 gwei.

  `21,000 * (10 + 2) = 252,000 gwei or 0.000252 ETH.`

When Jordan sends the money, 1.000252 ETH will be deducted from Jordan's account. Taylor will be credited 1.0000 ETH. Validator receives the tip of 0.000042 ETH. Base fee of 0.00021 ETH is burned.

The nice thing is that there are methods to determine the gas values for you so that you **do not need to manually calculate them.**

### Update the Transaction Object

The additional keys we need to add to our `txn_obj` are:
1. `gas` - in this simple ETH transfer case, that is 21,000 units but we can use the [`eth_estimateGas`](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.estimate_gas) RPC method to estimate this for a given transaction object. 

The next two offer some 'finer tuning' based on 1) how quickly and 2) how costly we want our transaction to be processed. We can provide **either one of these or both** for full control:

2. `maxPriorityFeePerGas` - this is the maximum 'tip' offered to the validator to process your transaction. We can use the [`eth_maxPriorityFeePerGas`](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.max_priority_fee) RPC method to return a suggested for a max priority fee. 

3. `maxFeePerGas` - This is the overall maximum fee paid per gas unit and includes the tip but no guarantee on *when* your transaction will be processed. Essentially:

  `maxFeePerGas = baseFeePerGas + maxPriorityFeePerGas`. 

Since we do not have an explicit method for the `maxFeePerGas`, we can manually determine this using the [`eth_getBlockByNumber`](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.get_block) RPC method for the `'pending'` block and extract the `baseFeePerGas` value. 

In [180]:
estimated_gas = w3_testnet.eth.estimate_gas(txn_obj)

In [181]:
max_priority_fee_per_gas = w3_testnet.eth.max_priority_fee

In [182]:
base_fee_per_gas = w3_testnet.eth.get_block("pending").baseFeePerGas

In [183]:
max_fee_per_gas = base_fee_per_gas + max_priority_fee_per_gas

In [184]:
txn_obj['gas'] = estimated_gas
txn_obj['maxPriorityFeePerGas'] = max_priority_fee_per_gas
txn_obj['maxFeePerGas'] = max_fee_per_gas

In [185]:
txn_obj

{'nonce': 3,
 'to': '0xc43ee4634Aed3796971ccE43e64b696D4de4ca83',
 'from': '0x3922C053bC7885B20F7550517E47BfCc692120Cc',
 'value': 10000000000000000,
 'data': b'',
 'chainId': 5,
 'gas': 21000,
 'maxPriorityFeePerGas': 1443,
 'maxFeePerGas': 1450}

Our transaction object is now ready to be signed. 

## Sign a Transaction

To sign a transaction with a private key, we use the [`eth_signTransaction`](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.sign_transaction) RPC method passing in both the `txn_obj` and the sender's `private_key`. 

In [186]:
signed_txn = w3_testnet.eth.account.sign_transaction(txn_obj, private_key)
signed_txn

SignedTransaction(rawTransaction=HexBytes('0x02f86d05038205a38205aa82520894c43ee4634aed3796971cce43e64b696d4de4ca83872386f26fc1000080c001a0dd7ac2982a5c4709c8544d9ca12d6d3076056e3733095bb969cdf589e7c79664a0186a107627561201eb165d8111894cf38a96f533e5246fc0f723e0cd57388561'), hash=HexBytes('0x53dd1b61fd0c12e2d49acac4d52db9b3eab39a0b29b136b43dec96814c2b6adf'), r=100178037919969082503890720786612462101050398138986304775701672550768545601124, s=11042907768222744092674064113870833103920562337777248885076738598081211696481, v=1)

## Send a Transaction

The last step is to send the transaction and broadcast it to the network. We use the [`eth_sendRawTransaction`](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.sign_transaction) RPC method passing in the `rawTransaction` value of the `signed_txn` object. If successful, the return value is the transaction hash (as a HexBytes object). 

In [187]:
txn_hash = w3_testnet.eth.send_raw_transaction(signed_txn.rawTransaction)
txn_hash

HexBytes('0x53dd1b61fd0c12e2d49acac4d52db9b3eab39a0b29b136b43dec96814c2b6adf')

We can now copy/paste the transaction hash in our Goerli block explorer https://goerli.etherscan.io and check our wallet to see that some test ETH from our wallet has indeed been removed. 

In [188]:
w3_testnet.eth.get_balance(public_address)

142724999908650000

## Summary

In this Code Along, we covered the fundamentals of creating a transaction on a public blockchain to send some test ETH tokens between wallet addresses. The process to create, sign and send raw transactions can be used for simple ETH transfers to more complex smart contract interactions and function calls as we will explore in the next set of Code Alongs. 